In [ ]:
import os
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StringType
import pandas as pd

# ========== Spark & Kafka Setup ==========
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 pyspark-shell'
findspark.init()

# === 1. Spark Initialization ===
spark = SparkSession.builder \
    .appName("CybersecurityLogAnalysis") \
    .getOrCreate()

# === 2. Define Kafka Log Schema ===
log_schema = StructType() \
    .add("timestamp", StringType()) \
    .add("source_ip", StringType()) \
    .add("destination_ip", StringType()) \
    .add("protocol", StringType()) \
    .add("port", StringType()) \
    .add("threat", StringType()) \
    .add("user_agent", StringType()) \
    .add("location", StringType()) \
    .add("bytes_sent", StringType()) \
    .add("bytes_received", StringType())

# === 3. Read Kafka Stream ===
raw_logs = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "logs") \
    .option("startingOffsets", "latest") \
    .load()

logs = raw_logs.selectExpr("CAST(value AS STRING) as json_str") \
    .select(from_json(col("json_str"), log_schema).alias("data")) \
    .select("data.*")

# === 4. Process Each Micro-Batch ===
def process_batch(batch_df, batch_id):
    print(f"📦 Processing batch {batch_id} with {batch_df.count()} records")
    batch_df.show(truncate=False)

# === 5. Start Streaming Query ===
query = logs.writeStream \
    .trigger(processingTime="30 seconds") \
    .foreachBatch(process_batch) \
    .outputMode("append") \
    .option("checkpointLocation", "/tmp/checkpoints/log_analysis") \
    .start()

query.awaitTermination()

📦 Processing batch 0 with 0 records
+---------+---------+--------------+--------+----+------+----------+--------+----------+--------------+
|timestamp|source_ip|destination_ip|protocol|port|threat|user_agent|location|bytes_sent|bytes_received|
+---------+---------+--------------+--------+----+------+----------+--------+----------+--------------+
+---------+---------+--------------+--------+----+------+----------+--------+----------+--------------+

📦 Processing batch 1 with 100 records
+-------------------+---------------+---------------+--------+----+----------+---------------------+--------+----------+--------------+
|timestamp          |source_ip      |destination_ip |protocol|port|threat    |user_agent           |location|bytes_sent|bytes_received|
+-------------------+---------------+---------------+--------+----+----------+---------------------+--------+----------+--------------+
|2025-12-18 17:10:03|107.106.84.81  |230.91.219.94  |UDP     |123 |none      |Python-urllib/3.8    |R